In [115]:
import numpy as np
import sys
import os
import argparse
import pickle
import fnmatch
from tqdm import tqdm
import re
import itertools
import random

ROOTDIR = '/mindhive/evlab/u/Shared/Greta_Eghbal_manifolds/'
model_identifier = 'NN-tree_nclass=64_nobj=64000_nhier=6_beta=0.000161_sigma=5.0_nfeat=936-train_test-fixed'
train_identifier = 'epochs-10_batch-32_lr-0.01_momentum-0.5_init-gaussian_std-1e-06'
analyze_identifier = 'mftma-exm_per_class=50-proj=False-rand=True-kappa=1e-08-n_t=300-n_rep=5'

layer = 'layer_2_Linear'
num_subsamples = 100

In [5]:
# Get extracted files for KNN sampling!
# /extracted/mftma-exm_per_class=50-proj=False-rand=True-kappa=1e-08-n_t=300-n_rep=5/NN-tree_nclass=64_nobj=64000_nhier=6_beta=0.000161_sigma=5.0_nfeat=936-train_test-fixed/epochs-10_batch-32_lr-0.01_momentum-0.5_init-gaussian_std-1e-06

In [31]:
extracted_files = []
for file in os.listdir(os.path.join(ROOTDIR, 'extracted', analyze_identifier, model_identifier, train_identifier)):
    if fnmatch.fnmatch(file, f'*{layer}_extracted.pkl'):
        extracted_files.append(os.path.join(ROOTDIR, 'extracted', analyze_identifier, model_identifier, train_identifier, file))


In [33]:
assert(len(extracted_files) == 1060)

In [34]:
s = [re.findall('/\d+', x) for x in extracted_files]
s = [item for sublist in s for item in sublist]
dummy_id = [(x.split('/')) for x in s]
file_id = [int(x.split('/')[1]) for x in s]
sorted_files = [extracted_files[x] for x in np.argsort(file_id)]

In [35]:
# Subsample
data = []
targets = []
epochs = []
subepochs = []

In [36]:
g = pickle.load(open(extracted_files[0],'rb'))
print(extracted_files[0])

/mindhive/evlab/u/Shared/Greta_Eghbal_manifolds/extracted/mftma-exm_per_class=50-proj=False-rand=True-kappa=1e-08-n_t=300-n_rep=5/NN-tree_nclass=64_nobj=64000_nhier=6_beta=0.000161_sigma=5.0_nfeat=936-train_test-fixed/epochs-10_batch-32_lr-0.01_momentum-0.5_init-gaussian_std-1e-06/0020_NN-tree_nclass=64_nobj=64000_nhier=6_beta=0.000161_sigma=5.0_nfeat=936-train_test-fixed-epoch=01-batchidx=315_layer_2_Linear_extracted.pkl


In [134]:
t = extracted_files[0].split('batchidx=')[1]
batchidx = t.split('_')[0]

In [137]:
t2 = extracted_files[0].split('epoch=')[1]
epoch = t2.split('-')[0]

In [138]:
epoch

'01'

In [37]:
for hier in range(len(g['projection_results'])):
    print(g['projection_results'][hier].keys())

dict_keys(['layer_2_Linear'])
dict_keys(['layer_2_Linear'])
dict_keys(['layer_2_Linear'])
dict_keys(['layer_2_Linear'])
dict_keys(['layer_2_Linear'])
dict_keys(['layer_2_Linear'])


In [40]:
len(g['projection_results'][hier][layer])

2

In [42]:
np.asarray(g['projection_results'][hier][layer]).shape

(2, 208, 50)

In [109]:
hier_idx = 4
projs = g['projection_results']
hier = np.asarray(projs[hier_idx][layer])

In [110]:
hier_moved = np.moveaxis(hier, 1, -1)

In [111]:
hier_moved.shape[0]

4

In [112]:
# reshape into # classes, # samples, # features
hier_moved = np.moveaxis(hier, 1, -1)
print(hier_moved.shape)

# flatten samples
hier_flat = np.reshape(hier_moved, [hier_moved.shape[0]*hier_moved.shape[1], hier_moved.shape[2]])
print(hier_flat.shape)

(4, 50, 208)
(200, 208)


In [113]:
# test flattening
print(f'Sample 1 from class 1: {hier_moved[0,0,:2]}')
print(f'Sample 2 from class 1: {hier_moved[0,1,:2]}')

print(f'Sample 1 from class 2: {hier_moved[1,0,:2]}')
print(f'Sample 2 from class 2: {hier_moved[1,1,:2]}')

print(f'Sample 1 flattened: {hier_flat[0][:2]}')
print(f'Sample 2 flattened: {hier_flat[1][:2]}')
print(f'Sample 51 flattened: {hier_flat[50][:2]}')
print(f'Sample 52 flattened: {hier_flat[51][:2]}')
# ie the first 50 samples (rows) correspond to class 1, the next 50 rows for class 2 etc

Sample 1 from class 1: [ 9.2271031e-05 -4.1967232e-04]
Sample 2 from class 1: [-0.00041304 -0.00055493]
Sample 1 from class 2: [-0.00034093  0.00033659]
Sample 2 from class 2: [-0.00067744 -0.00057679]


In [119]:
targets_array = np.asarray([[(i+1)]*hier_moved.shape[1] for i in range(hier_moved.shape[0])])

In [139]:
targets_flatten = targets_array.ravel()

In [125]:
sample_idx = random.sample(range(hier_flat.shape[0]), num_subsamples)

subsampled_data = hier_flat[sample_idx, :]
subsampled_targets = targets_flatten[sample_idx]

assert(len(subsampled_data) == num_subsamples)

In [140]:
data.append(subsampled_data)
targets.append(targets_flatten)
epochs.append(epoch*num_subsamples)
subepochs.append(subepoch*num_subsamples)

NameError: name 'data' is not defined

In [141]:
production_time = list(range(1,len(epoch))) 
time = list(range(1,len(sorted_files)))


SyntaxError: invalid syntax (<ipython-input-141-3e827e153922>, line 1)

In [ ]:
relTime = productionTime./max(productionTime);

In [143]:
time = list(range(1,len(sorted_files)))
